In [ ]:
import random
import pandas as pd
import plotly.express as px

# --- Simulation Configuration ---
NUM_SIMULATIONS = 10000  # Number of characters to simulate for each method
MAX_LEVEL = 20
HIT_DIE = 10
CON_MODIFIER = 3  # Example: Constitution score of 16-17
# -----------------------------

# Calculate the average HP gain per level (half of hit die + 1)
AVERAGE_ROLL = HIT_DIE // 2 + 1
AVERAGE_HP_GAIN = AVERAGE_ROLL + CON_MODIFIER

print(f"Simulating {NUM_SIMULATIONS} characters up to level {MAX_LEVEL}.")
print(f"Hit Die: d{HIT_DIE}")
print(f"Constitution Modifier: +{CON_MODIFIER}")
print(f"Average HP Gain per level (after level 1): {AVERAGE_HP_GAIN}")

def get_baseline_hp():
    """Calculates the HP progression taking the average every level."""
    hp_by_level = {}
    # Level 1 HP
    hp_by_level[1] = HIT_DIE + CON_MODIFIER
    # Levels 2-20
    for level in range(2, MAX_LEVEL + 1):
        hp_by_level[level] = hp_by_level[level - 1] + AVERAGE_HP_GAIN
    return hp_by_level

BASELINE_HP = get_baseline_hp()

def simulate_hp_rolling():
    """Simulates HP progression by rolling a hit die at each level."""
    hp_by_level = {}
    # Level 1 HP is always max
    hp_by_level[1] = HIT_DIE + CON_MODIFIER
    # Levels 2-20
    for level in range(2, MAX_LEVEL + 1):
        roll = random.randint(1, HIT_DIE)
        hp_by_level[level] = hp_by_level[level - 1] + roll + CON_MODIFIER
    return hp_by_level

def simulate_hp_rerolling(ahead_levels_threshold):
    """
    Simulates HP progression with a reroll rule.
    If the character's HP is not ahead of the average curve by a certain number
    of levels, all previous dice are rerolled once for that level. Once they are
    ahead, they take the average for all subsequent levels.
    """
    hp_by_level = {}
    is_ahead = False
    
    # Level 1
    hp_by_level[1] = HIT_DIE + CON_MODIFIER
    rolls = [HIT_DIE]

    for level in range(2, MAX_LEVEL + 1):
        if is_ahead:
            # If we are already ahead, just take the average
            hp_by_level[level] = hp_by_level[level - 1] + AVERAGE_HP_GAIN
            continue

        # Add a roll for the current level
        rolls.append(random.randint(1, HIT_DIE))
        
        # The target level to compare against
        target_level = min(MAX_LEVEL, level + ahead_levels_threshold)
        
        # Check if we need to reroll (only once per level)
        current_total_hp = sum(rolls) + (level * CON_MODIFIER)
        if current_total_hp < BASELINE_HP[target_level]:
            # Reroll all dice once for this level
            rolls = [random.randint(1, HIT_DIE) for _ in range(level)]
            # Level 1 is always max, so fix the first die
            rolls[0] = HIT_DIE

        # Recalculate HP progression up to the current level based on the final rolls
        current_hp = 0
        for i in range(level):
            current_hp += rolls[i] + CON_MODIFIER
            hp_by_level[i + 1] = current_hp
            
        # Check if we are now ahead of the curve after this level's roll/reroll
        final_total_hp = sum(rolls) + (level * CON_MODIFIER)
        if final_total_hp >= BASELINE_HP[target_level]:
            is_ahead = True

    return hp_by_level

# --- Run Simulations ---
all_results = []

# Simulate each method
for i in range(NUM_SIMULATIONS):
    # Baseline Average
    hp_avg = get_baseline_hp()
    for level, hp in hp_avg.items():
        all_results.append({'simulation': i, 'level': level, 'hp': hp, 'method': 'Average'})

    # Standard Rolling
    hp_roll = simulate_hp_rolling()
    for level, hp in hp_roll.items():
        all_results.append({'simulation': i, 'level': level, 'hp': hp, 'method': 'Rolling'})

    # Reroll if not 1 level ahead
    hp_reroll_1 = simulate_hp_rerolling(ahead_levels_threshold=1)
    for level, hp in hp_reroll_1.items():
        all_results.append({'simulation': i, 'level': level, 'hp': hp, 'method': 'Reroll if not 1 Level Ahead'})
        
    # Reroll if not 2 levels ahead
    hp_reroll_2 = simulate_hp_rerolling(ahead_levels_threshold=2)
    for level, hp in hp_reroll_2.items():
        all_results.append({'simulation': i, 'level': level, 'hp': hp, 'method': 'Reroll if not 2 Levels Ahead'})

# Create a DataFrame
df = pd.DataFrame(all_results)

print("Simulations complete. DataFrame created.")
df.head()

# --- Analyze and Visualize the Results ---

# Calculate the average HP at each level for each method
df_avg_hp = df.groupby(['method', 'level'])['hp'].mean().reset_index()

# Plot the average HP progression
fig = px.line(
    df_avg_hp,
    x='level',
    y='hp',
    color='method',
    title='Average HP Progression by Method',
    labels={'level': 'Character Level', 'hp': 'Average Hit Points'},
    template='plotly_white'
)

fig.update_layout(
    legend_title_text='HP Method',
    xaxis=dict(tickmode='linear', dtick=1)
)

fig.show()

## Analysis of Being "Ahead of the Curve"

Now, let's analyze the probability that a character using a particular HP method will be "ahead of the curve." We'll define this as having more HP than a character who takes the average HP at a higher level. This gives us an idea of the method's tendency to produce high-HP characters.

We will calculate two metrics for each method at each level:
1.  **Chance to be 1 Level Ahead:** The percentage of simulated characters whose HP is greater than the baseline average HP of a character one level higher.
2.  **Chance to be 2 Levels Ahead:** The percentage of simulated characters whose HP is greater than the baseline average HP of a character two levels higher.

def calculate_ahead_chance(df, levels_ahead):
    """
    Calculates the percentage chance of being ahead of the baseline HP curve.
    """
    ahead_results = []
    
    # We can't calculate for levels that would exceed the max level
    for level in range(1, MAX_LEVEL + 1 - levels_ahead):
        # Get the target HP to beat
        target_hp = BASELINE_HP[level + levels_ahead]
        
        for method in df['method'].unique():
            if method == 'Average': # The average can never be ahead of itself
                chance = 0
            else:
                # Get all HP values for the current level and method
                level_data = df[(df['level'] == level) & (df['method'] == method)]
                
                # Count how many are ahead of the target HP
                is_ahead_count = (level_data['hp'] > target_hp).sum()
                
                # Calculate the percentage
                chance = (is_ahead_count / len(level_data)) * 100
            
            ahead_results.append({
                'level': level,
                'method': method,
                'chance': chance,
                'ahead_by': f'{levels_ahead} Level(s)'
            })
            
    return pd.DataFrame(ahead_results)

# Calculate the chances for being 1 and 2 levels ahead
df_ahead_1 = calculate_ahead_chance(df, 1)
df_ahead_2 = calculate_ahead_chance(df, 2)

# Combine the results
df_ahead = pd.concat([df_ahead_1, df_ahead_2])

# Plot the results
fig_ahead = px.line(
    df_ahead,
    x='level',
    y='chance',
    color='method',
    facet_row='ahead_by',
    title='Probability of Being Ahead of the Average HP Curve',
    labels={'level': 'Character Level', 'chance': 'Probability (%)'},
    template='plotly_white',
    height=800
)

fig_ahead.update_yaxes(matches=None)
fig_ahead.update_layout(
    legend_title_text='HP Method',
    xaxis=dict(tickmode='linear', dtick=1)
)

fig_ahead.show()

## Summary Table: Chance to Be Ahead of the Curve

The table below summarizes the analysis, showing the probability for each method to be 1 or 2 levels ahead of the baseline average HP at key character levels. This provides a direct comparison of how likely each method is to produce a high-HP character at different stages of an adventurer's career.

# --- Create a Summary Table ---

# Pivot the table to get the desired format for all levels
summary_pivot = df_ahead.pivot_table(
    index='method',
    columns=['ahead_by', 'level'],
    values='chance'
)

# Clean up the table for presentation
# Reorder the top-level columns if they exist
if '1 Level(s)' in summary_pivot.columns.get_level_values(0):
    summary_pivot = summary_pivot.reindex(columns=['1 Level(s)', '2 Level(s)'], level=0)

# Sort the second-level columns (levels)
summary_pivot = summary_pivot.sort_index(axis=1, level=1)

# Display the formatted table, dropping columns that are all NaN 
# (e.g., levels 19 & 20 for '2 Levels Ahead' cannot be calculated)
summary_pivot.dropna(axis=1, how='all').style.format("{:.2f}%").set_caption("Percentage Chance to Be Ahead of the HP Curve at All Levels")
